# 通过示例学习PyTorch

PyTorch的核心是提供两个主要功能：
1. n维张量，类似于Numpy，但可以在GPU上运行
2. 用于构建和训练神经网络的自动微分

将使用三阶多项式拟合`y=sin(x)`的问题作为运行示例。

该网络将具有四个参数，并且将通过使网络输出与实际输出之间的欧几里得距离最小化来进行梯度下降训练，以适应随机数据。

# 张量
## 预热：Numpy
介绍PyTorch之前，首先使用Numpy实现网络。

Numpy提供了一个n维数组对象，以及许多用于操纵这些数组的函数。

Numpy是用于科学计算的通用框架。它对计算图，深度学习或梯度一无所知。但是，通过使用Numpy操作手动实现网络的前向和后向传递，我们可以轻松地使用Numpy使三阶多项式适合正弦函数。

In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import math

# Create random input and output data
x = np.linspace(-math.pi, math.pi, 2000)
y = np.sin(x)

# Randomly initialize weights
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    # y = a + b x + c x^2 + d x^3
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Result: y = {a} + {b} x + {c} x^2 + {d} x^3')

99 1495.428321453981
199 1060.764042568304
299 753.1924021114385
399 535.5514103751552
499 381.54568204649775
599 272.5687168956434
699 195.45456425763035
799 140.8869560817325
899 102.27364031351541
999 74.94987554878297
1099 55.61483180669906
1199 41.93279095575555
1299 32.250956846046066
1399 25.399777216353858
1499 20.551650400326615
1599 17.120944973670223
1699 14.693252492207215
1799 12.975325705509636
1899 11.759653980173438
1999 10.899396175678932
Result: y = -0.04829458833822945 + 0.8574307000490787 x + 0.00833162011376332 x^2 + -0.09342851518800707 x^3


## PyTorch张量
Numpy是一个很棒的框架，但是它不能利用GPU来加速其数值计算。

对于现代深度神经网络，GPU通常会提供50倍或者更高的加速，因此Numpy不足以实现现代深度学习。

PyTorch张量在概念上与Numpy数组相同：张量是n维数组，PyTorch提供了许多在这些张量上进行操作的函数。在幕后，张量可以跟踪计算图和梯度，但它们也可用作科学计算的通用工具。

要在GPU上运行PyTorch张量，只需要指定正确的设备即可。

此处，使用PyTorch张量将三阶多项式拟合为正弦函数。

此处，使用PyTorch张量将三阶多项式拟合为正弦函数。

需要手动实现通过网络的正向和反向传播

In [2]:
# -*- coding: utf-8 -*-

import torch
import math

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# Create random input and output data
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Randomly initialize weights
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights using gradient descent
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 3419.602783203125
199 2265.134765625
299 1501.469970703125
399 996.3026123046875
499 662.12353515625
599 441.0506896972656
699 294.79766845703125
799 198.0392608642578
899 134.02352905273438
999 91.66876220703125
1099 63.644378662109375
1199 45.10109329223633
1299 32.83070755004883
1399 24.710803985595703
1499 19.337257385253906
1599 15.780961990356445
1699 13.427223205566406
1799 11.869264602661133
1899 10.837987899780273
1999 10.1552734375
Result: y = -0.005623095203191042 + 0.8215587735176086 x + 0.0009700797381810844 x^2 + -0.08832604438066483 x^3


# Autograd
## PyTorch：张量和Autograd
上述示例中，通过手动实现神经网络的前向和后向传递，对于大型复杂的网络来说，会变得很麻烦。

幸运的是，可以使用自动微分来自动计算神经网络中的反向传递。

PyTorch中的**Autograd**包提供了此功能。

使用Autograd时，网络的正向传播将定义计算图，图中的节点为张量，边为从输入张量产生输出张量的函数，之后通过该图进行反向传播，可以轻松计算梯度。

每个张量代表计算图中的一个节点，如果`x`是具有`x.requires_grad=True`的张量，则`x.grad`是另一个张量，其保持`x`相对于某个标量值的梯度。

此处，使用PyTorch张量和Autograd来实现我们的正弦波与三阶多项式示例，现在我们不再需要通过网络手动实现反向传递

In [6]:
# -*- coding: utf-8 -*-
import torch
import math

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")  # Uncomment this to run on GPU

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For a third order polynomial, we need
# 4 weights: y = a + b x + c x^2 + d x^3
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y using operations on Tensors.
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
    # the gradient of the loss with respect to a, b, c, d respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 175.24197387695312
199 123.08644104003906
299 87.36510467529297
399 62.8709831237793
499 46.056190490722656
599 34.500152587890625
699 26.5491886138916
799 21.072628021240234
899 17.296268463134766
999 14.689461708068848
1099 12.888094902038574
1199 11.6420316696167
1299 10.779184341430664
1399 10.181126594543457
1499 9.766201972961426
1599 9.478055953979492
1699 9.2777738571167
1799 9.13843822479248
1899 9.041420936584473
1999 8.973814964294434
Result: y = -0.011952102184295654 + 0.8514927625656128 x + 0.0020619353745132685 x^2 + -0.09258389472961426 x^3


## PyTorch：定义新的Autograd函数
在背后，每个原始的Autograd运算符实际上都是在张量上运行的两个函数。
1. 正向函数从输入张量计算输出张量。
2. 反向函数接收相对于某个标量值的输出张量的梯度，并计算相对于相同标量值的输入张量的梯度。

PyTorch中，可以通过定义`torch.autograd.Function`的子类并实现`forward`和`backward`函数来轻松定义自己的Autograd运算符。

之后通过构造实例，并像调用函数一样调用新的Autograd运算符，并传递包含输入数据的张量。

在此示例中，我们将模型定义为`y=a+b p[3](c+dx)`而不是`y=a+bx+cx^2+dx^3`，其中`P[3](x)=1/2(5x^3-3x)`是三次勒让德多项式。

编写了自己定义的Autograd函数来计算P[3]的`forward`和`backward`，并使用它来实现我们的模型

In [7]:
# -*- coding: utf-8 -*-
import torch
import math

class LegendrePolynomial3(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        return 0.5 * (5 * input ** 3 - 3 * input)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        return grad_output * 1.5 * (5 * input ** 2 - 1)

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")  # Uncomment this to run on GPU

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For this example, we need
# 4 weights: y = a + b * P3(c + d * x), these weights need to be initialized
# not too far from the correct result to ensure convergence.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
b = torch.full((), -1.0, device=device, dtype=dtype, requires_grad=True)
c = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
d = torch.full((), 0.3, device=device, dtype=dtype, requires_grad=True)

learning_rate = 5e-6
for t in range(2000):
    # To apply our Function, we use Function.apply method. We alias this as 'P3'.
    P3 = LegendrePolynomial3.apply

    # Forward pass: compute predicted y using operations; we compute
    # P3 using our custom autograd operation.
    y_pred = a + b * P3(c + d * x)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass.
    loss.backward()

    # Update weights using gradient descent
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} * P3({c.item()} + {d.item()} x)')

99 209.95834350585938
199 144.66018676757812
299 100.70249938964844
399 71.03519439697266
499 50.97850799560547
599 37.403133392333984
699 28.206867218017578
799 21.973188400268555
899 17.7457275390625
999 14.877889633178711
1099 12.931766510009766
1199 11.610918045043945
1299 10.714258193969727
1399 10.10548210144043
1499 9.692106246948242
1599 9.411375999450684
1699 9.220745086669922
1799 9.091285705566406
1899 9.003361701965332
1999 8.943639755249023
Result: y = -6.8844756562214116e-09 + -2.208526849746704 * P3(1.5037101563919464e-09 + 0.2554861009120941 x)


# `nn`模块
计算图和Autograd是定义复杂运算符并自动采用导数的非常强大的范例。
但是对于大型神经网络，原始的Autograd可能会太低级。

在构建神经网络时，我们经常想到将计算安排在`层`中，其中某些据有`可学习的参数`，这些参数会在学习期间进行优化。

在PyTorch中，`nn`包也达到了相同的目的。`nn`包定义了一组模块，它们大致等效于神经网络层。模块接收输入张量并计算输出张量，但也可以保持内部状态，例如包含可学习参数的张量。`nn`包还定义了一组有用的损失函数，这些函数通常在训练神经网络时使用。
在此示例中，我们可以使用`nn`包来实现我们多项式模型网络

In [1]:
# -*- coding: utf-8 -*-
import torch
import math

# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# For this example, the output y is a linear function of (x, x^2, x^3), so
# we can consider it as a linear layer neural network. Let's prepare the
# tensor (x, x^2, x^3).
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# In the above code, x.unsqueeze(-1) has shape (2000, 1), and p has shape
# (3,), for this case, broadcasting semantics will apply to obtain a tensor
# of shape (2000, 3) 

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. The Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
# The Flatten layer flatens the output of the linear layer to a 1D tensor,
# to match the shape of `y`.
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
for t in range(2000):

    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(xx)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

# You can access the first layer of `model` like accessing the first item of a list
linear_layer = model[0]

# For linear layer, its parameters are stored as `weight` and `bias`.
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 1539.209228515625
199 1029.7877197265625
299 690.3020629882812
399 463.95538330078125
499 312.9661865234375
599 212.19236755371094
699 144.89608764648438
799 99.92948150634766
899 69.8648452758789
999 49.75074005126953
1099 36.28491973876953
1199 27.26357078552246
1299 21.21539306640625
1399 17.1574764251709
1499 14.432598114013672
1599 12.601414680480957
1699 11.369772911071777
1799 10.540678024291992
1899 9.982046127319336
1999 9.60525894165039
Result: y = -0.017449673265218735 + 0.8346524238586426 x + 0.0030103588942438364 x^2 + -0.09018850326538086 x^3


## PyTorch：`optim`

到目前为止，我们已经通过使用`torch.no_grad()`手动更改持有可学习参数的张量来更新模型的权重。

对于像随机梯度下降这样的简单优化算法来说，这并不是一个巨大的负担，但是在实践中，我们经常使用更复杂的优化器（例如AdaGrad、RMSProp、Adam等）来训练神经网络。

PyTorch中的`optim`包抽象了优化算法的思想，并提供了常用优化算法的实现。

在此示例中，我们将使用`nn`包像以前一样定义我们的模型，但是我们将使用`optim`包提供的RMSprop算法来优化模型。

In [5]:
# -*- coding: utf-8 -*-
import torch
import math

# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Prepare the input tensor (x, x^2, x^3).
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use RMSprop; the optim package contains many other
# optimization algorithms. The first argument to the RMSprop constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
for t in range(2000):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(xx)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

linear_layer = model[0]
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 26487.431640625
199 12990.53125
299 6329.84375
399 3461.06884765625
499 2551.99658203125
599 2248.423828125
699 1993.4912109375
799 1734.332275390625
899 1487.4154052734375
999 1260.345458984375
1099 1054.623779296875
1199 869.8262939453125
1299 705.1729125976562
1399 559.9568481445312
1499 433.5810852050781
1599 325.52691650390625
1699 235.22348022460938
1799 162.3294219970703
1899 105.24195861816406
1999 63.403404235839844
Result: y = -0.0005081858835183084 + 0.6301236748695374 x + -0.0005082738935016096 x^2 + -0.06181204319000244 x^3


## PyTorch：自定义`nn`模块

有时，需要指定比一系列现有模块更复杂的模型。对于这些情况，可以通过子类化`nn.Module`并定义一个`forward`来定义自己的模块，该模块使用其他模块或在Tensors上的其他自动转换操作来接收输入Tensors并生成输出Tensors。

在此示例中，我们将三阶多项式实现为自定义`Module`子类

In [6]:
# -*- coding: utf-8 -*-
import torch
import math

class Polynomial3(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate four parameters and assign them as
        member parameters.
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        return self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3

    def string(self):
        """
        Just like any class in Python, you can also define custom method on PyTorch modules
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3'

# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Construct our model by instantiating the class defined above
model = Polynomial3()

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the nn.Linear
# module which is members of the model.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)
for t in range(2000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')

99 9.078651428222656
199 8.989288330078125
299 8.93113899230957
399 8.892642974853516
499 8.867155075073242
599 8.850278854370117
699 8.839099884033203
799 8.83169937133789
899 8.826797485351562
999 8.823548316955566
1099 8.821395874023438
1199 8.819971084594727
1299 8.819026947021484
1399 8.818400382995605
1499 8.817984580993652
1599 8.817709922790527
1699 8.817527770996094
1799 8.817405700683594
1899 8.817325592041016
1999 8.817272186279297
Result: y = -8.687808440299705e-05 + 0.8564354181289673 x + 1.4987917893449776e-05 x^2 + -0.09328693896532059 x^3


## PyTorch：控制流+权重共享

作为动态图和权重共享的示例，实现了一个非常奇怪的模型：

一个三阶多项式，在每个正向传播中选择3到5之间的一个随机数，并使用该阶数，多次使用相同的权重重复计算四和五阶。

对于此模型，我们可以使用常规的Python流控制来实现循环，并且可以通过在定义正向传播时简单地多次重复使用相同的参数来实现权重共享。

可以轻松地将此模型实现为`Module`子类

In [7]:
# -*- coding: utf-8 -*-
import random
import torch
import math

class DynamicNet(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate five parameters and assign them as members.
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        self.e = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 4, 5
        and reuse the e parameter to compute the contribution of these orders.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same parameter many
        times when defining a computational graph.
        """
        y = self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3
        for exp in range(4, random.randint(4, 6)):
            y = y + self.e * x ** exp
        return y

    def string(self):
        """
        Just like any class in Python, you can also define custom method on PyTorch modules
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3 + {self.e.item()} x^4 ? + {self.e.item()} x^5 ?'

# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Construct our model by instantiating the class defined above
model = DynamicNet()

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)
for t in range(30000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 2000 == 1999:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')

1999 486.6675109863281
3999 237.21131896972656
5999 125.17976379394531
7999 63.94478988647461
9999 34.18781661987305
11999 21.14102554321289
13999 14.75257396697998
15999 11.643495559692383
17999 10.224358558654785
19999 9.371317863464355
21999 8.963484764099121
23999 8.979628562927246
25999 8.649065017700195
27999 8.646810531616211
29999 8.849112510681152
Result: y = -0.003552334615960717 + 0.8557438254356384 x + 0.00015717394126113504 x^2 + -0.09339725226163864 x^3 + 7.602591358590871e-05 x^4 ? + 7.602591358590871e-05 x^5 ?
